# 🚀 MLPY Framework - Demostración Completa

## California Housing Price Prediction

Este notebook demuestra las capacidades completas del framework MLPY a través de un problema real: la predicción de precios de viviendas en California.

### 📋 Contenido

1. **Carga y Validación de Datos**
2. **Creación de Tareas**
3. **Modelos Base**
4. **Modelos Avanzados**
5. **Ensemble Learning**
6. **Cross-Validation**
7. **Model Registry**
8. **Comparación con scikit-learn**
9. **Visualizaciones**
10. **Conclusiones**

## 📦 Importaciones

In [ ]:
# Configuración inicial
import warnings
warnings.filterwarnings('ignore')

# Librerías estándar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from IPython.display import display, Markdown, HTML

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías base cargadas")

In [ ]:
# MLPY imports
from mlpy.tasks import TaskRegr
from mlpy.learners.baseline import LearnerRegrFeatureless
from mlpy.learners.sklearn import (
    LearnerLinearRegression,
    LearnerRandomForestRegressor,
    LearnerGradientBoostingRegressor
)
from mlpy.learners.ensemble import LearnerVoting
from mlpy.resamplings import ResamplingCV, ResamplingHoldout
from mlpy.measures import MeasureRegrMSE, MeasureRegrMAE
from mlpy.validation.validators import validate_task_data
from mlpy.model_registry.registry import ModelRegistry, ModelMetadata
from mlpy.model_registry.registry import ModelCategory, TaskType, Complexity

print("✅ MLPY framework cargado")
print(f"📍 Versión: 2.1")

## 1️⃣ Carga y Exploración de Datos

In [ ]:
# Cargar el dataset de California Housing
from sklearn.datasets import fetch_california_housing

print("📊 Cargando California Housing Dataset...")
housing = fetch_california_housing(as_frame=True)
data = housing.frame

# Información básica
display(Markdown("### 📈 Información del Dataset"))
print(f"• Forma: {data.shape}")
print(f"• Características: {list(housing.feature_names)}")
print(f"• Variable objetivo: MedHouseVal (Valor medio de vivienda en $100,000s)")
print(f"• Registros: {len(data):,}")
print(f"• Sin valores faltantes: {data.isnull().sum().sum() == 0}")

In [ ]:
# Vista previa de los datos
display(Markdown("### 🔍 Primeras 5 filas"))
display(data.head())

display(Markdown("### 📊 Estadísticas descriptivas"))
display(data.describe().round(2))

In [ ]:
# Visualización de distribuciones
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
fig.suptitle('Distribución de Variables', fontsize=16, y=1.02)

for idx, col in enumerate(data.columns):
    ax = axes[idx // 3, idx % 3]
    data[col].hist(bins=30, ax=ax, edgecolor='black', alpha=0.7)
    ax.set_title(col)
    ax.set_xlabel('Valor')
    ax.set_ylabel('Frecuencia')
    
    # Añadir estadísticas
    mean_val = data[col].mean()
    median_val = data[col].median()
    ax.axvline(mean_val, color='red', linestyle='--', alpha=0.5, label=f'Media: {mean_val:.2f}')
    ax.axvline(median_val, color='green', linestyle='--', alpha=0.5, label=f'Mediana: {median_val:.2f}')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

## 2️⃣ Validación de Datos con MLPY

In [ ]:
# Usar el sistema de validación de MLPY
display(Markdown("### 🔍 Validación Automática de Datos"))

validation_result = validate_task_data(
    data, 
    target='MedHouseVal',
    task_type='regression'
)

# Mostrar resultados de validación
status_emoji = "✅" if validation_result['valid'] else "❌"
print(f"\n{status_emoji} Estado de Validación: {'APROBADO' if validation_result['valid'] else 'FALLIDO'}")
print(f"📊 Número de muestras: {validation_result['stats']['n_samples']:,}")
print(f"🔢 Número de características: {validation_result['stats']['n_features']}")

if validation_result.get('warnings'):
    display(Markdown("### ⚠️ Advertencias"))
    for warning in validation_result['warnings']:
        print(f"  • {warning}")

if validation_result.get('suggestions'):
    display(Markdown("### 💡 Sugerencias"))
    for suggestion in validation_result['suggestions']:
        print(f"  • {suggestion}")

## 3️⃣ Creación de Tarea MLPY

In [ ]:
# Crear tarea de regresión
display(Markdown("### 🎯 Creación de Tarea de Regresión"))

task = TaskRegr(data=data, target='MedHouseVal')

print(f"✅ Tarea creada: {task}")
print(f"📋 Tipo de tarea: {task.task_type}")
print(f"📊 Observaciones: {task.nrow:,}")
print(f"🔢 Características: {task.ncol - 1}")
print(f"📝 Nombres de características: {task.feature_names}")

In [ ]:
# Crear división train/test
display(Markdown("### 🔀 División Train/Test (80/20)"))

holdout = ResamplingHoldout(ratio=0.2, stratify=False)
holdout_instance = holdout.instantiate(task)

train_idx = holdout_instance.train_set(0)
test_idx = holdout_instance.test_set(0)

train_task = task.filter(train_idx)
test_task = task.filter(test_idx)

# Mostrar información de la división
train_size = len(train_idx)
test_size = len(test_idx)
total = train_size + test_size

print(f"📚 Conjunto de entrenamiento: {train_size:,} ({train_size/total*100:.1f}%)")
print(f"🧪 Conjunto de prueba: {test_size:,} ({test_size/total*100:.1f}%)")
print(f"📊 Total: {total:,}")

## 4️⃣ Modelo Base (Baseline)

In [ ]:
display(Markdown("### 📏 Entrenamiento del Modelo Baseline"))
print("El modelo baseline predice siempre la media del conjunto de entrenamiento.\n")

# Entrenar modelo baseline
baseline = LearnerRegrFeatureless(id="baseline")

start_time = time()
baseline.train(train_task)
baseline_train_time = time() - start_time

# Predicciones
baseline_pred = baseline.predict(test_task)

# Evaluación
mse_measure = MeasureRegrMSE()
mae_measure = MeasureRegrMAE()

baseline_mse = mse_measure.score(baseline_pred.truth, baseline_pred.response)
baseline_mae = mae_measure.score(baseline_pred.truth, baseline_pred.response)

# Mostrar resultados
results_html = f"""
<div style="background-color: #f0f0f0; padding: 15px; border-radius: 10px; margin: 10px 0;">
    <h4>📊 Resultados del Modelo Baseline</h4>
    <ul style="list-style-type: none;">
        <li>📉 <b>MSE:</b> {baseline_mse:.4f}</li>
        <li>📈 <b>MAE:</b> {baseline_mae:.4f}</li>
        <li>⏱️ <b>Tiempo de entrenamiento:</b> {baseline_train_time:.3f}s</li>
    </ul>
</div>
"""
display(HTML(results_html))

## 5️⃣ Modelos Avanzados con MLPY

In [ ]:
display(Markdown("### 🚀 Entrenamiento de Modelos Avanzados"))

# Definir modelos a probar
models = {
    'Linear Regression': LearnerLinearRegression(),
    'Random Forest': LearnerRandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': LearnerGradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = {}

# Entrenar y evaluar cada modelo
for name, learner in models.items():
    print(f"\n🔄 Entrenando {name}...")
    
    # Entrenar
    start_time = time()
    learner.train(train_task)
    train_time = time() - start_time
    
    # Predecir
    predictions = learner.predict(test_task)
    
    # Evaluar
    mse = mse_measure.score(predictions.truth, predictions.response)
    mae = mae_measure.score(predictions.truth, predictions.response)
    
    results[name] = {
        'mse': mse,
        'mae': mae,
        'train_time': train_time,
        'predictions': predictions
    }
    
    # Mostrar resultados
    improvement = ((baseline_mse - mse) / baseline_mse) * 100
    print(f"  ✅ MSE: {mse:.4f} (mejora: {improvement:.1f}% vs baseline)")
    print(f"  ✅ MAE: {mae:.4f}")
    print(f"  ⏱️ Tiempo: {train_time:.3f}s")

## 6️⃣ Ensemble Learning

In [ ]:
display(Markdown("### 🎭 Creación de Ensemble con Voting"))

# Crear ensemble de votación
base_learners = [
    LearnerLinearRegression(id="lr"),
    LearnerRandomForestRegressor(n_estimators=50, random_state=42, id="rf"),
    LearnerGradientBoostingRegressor(n_estimators=50, random_state=42, id="gb")
]

print("📦 Componentes del ensemble:")
print("  • Linear Regression (peso: 0.2)")
print("  • Random Forest (peso: 0.4)")
print("  • Gradient Boosting (peso: 0.4)")

voting_ensemble = LearnerVoting(
    base_learners=base_learners,
    voting='soft',
    weights=[0.2, 0.4, 0.4]
)

# Entrenar ensemble
print("\n🔄 Entrenando ensemble...")
start_time = time()
voting_ensemble.train(train_task)
ensemble_train_time = time() - start_time

# Predecir
ensemble_pred = voting_ensemble.predict(test_task)

# Evaluar
ensemble_mse = mse_measure.score(ensemble_pred.truth, ensemble_pred.response)
ensemble_mae = mae_measure.score(ensemble_pred.truth, ensemble_pred.response)

results['Voting Ensemble'] = {
    'mse': ensemble_mse,
    'mae': ensemble_mae,
    'train_time': ensemble_train_time,
    'predictions': ensemble_pred
}

# Mostrar resultados
improvement = ((baseline_mse - ensemble_mse) / baseline_mse) * 100
print(f"\n✅ Ensemble MSE: {ensemble_mse:.4f} (mejora: {improvement:.1f}% vs baseline)")
print(f"✅ Ensemble MAE: {ensemble_mae:.4f}")
print(f"⏱️ Tiempo de entrenamiento: {ensemble_train_time:.3f}s")

## 7️⃣ Cross-Validation

In [ ]:
display(Markdown("### 🔄 5-Fold Cross-Validation"))

# Encontrar el mejor modelo
best_model_name = min(results.keys(), key=lambda x: results[x]['mse'])
print(f"🏆 Mejor modelo hasta ahora: {best_model_name}")
print(f"   MSE: {results[best_model_name]['mse']:.4f}")

# Realizar cross-validation
print("\n🔄 Ejecutando 5-fold cross-validation...\n")
cv = ResamplingCV(folds=5)
cv_instance = cv.instantiate(task)

cv_scores = []
for fold in range(5):
    # Obtener datos del fold
    train_idx = cv_instance.train_set(fold)
    test_idx = cv_instance.test_set(fold)
    
    fold_train = task.filter(train_idx)
    fold_test = task.filter(test_idx)
    
    # Entrenar modelo
    if best_model_name == 'Random Forest':
        model = LearnerRandomForestRegressor(n_estimators=100, random_state=42)
    elif best_model_name == 'Gradient Boosting':
        model = LearnerGradientBoostingRegressor(n_estimators=100, random_state=42)
    elif best_model_name == 'Voting Ensemble':
        model = voting_ensemble
    else:
        model = LearnerLinearRegression()
    
    model.train(fold_train)
    
    # Predecir y evaluar
    pred = model.predict(fold_test)
    fold_mse = mse_measure.score(pred.truth, pred.response)
    cv_scores.append(fold_mse)
    
    print(f"  Fold {fold+1}: MSE = {fold_mse:.4f}")

# Resultados de CV
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

display(HTML(f"""
<div style="background-color: #e8f5e9; padding: 15px; border-radius: 10px; margin: 15px 0;">
    <h4>📊 Resultados de Cross-Validation</h4>
    <ul style="list-style-type: none;">
        <li>📈 <b>MSE Promedio:</b> {cv_mean:.4f}</li>
        <li>📉 <b>Desviación Estándar:</b> {cv_std:.4f}</li>
        <li>🎯 <b>Intervalo de Confianza (95%):</b> [{cv_mean - 1.96*cv_std:.4f}, {cv_mean + 1.96*cv_std:.4f}]</li>
    </ul>
</div>
"""))

## 8️⃣ Model Registry

In [ ]:
display(Markdown("### 📚 Registro del Mejor Modelo"))

# Inicializar registro
registry = ModelRegistry()

# Crear metadata del modelo
metadata = ModelMetadata(
    name="housing_price_predictor",
    display_name="California Housing Price Predictor",
    description=f"Mejor modelo ({best_model_name}) para predicción de precios de vivienda en California",
    category=ModelCategory.TRADITIONAL_ML,
    class_path=f"mlpy.learners.sklearn.Learner{best_model_name.replace(' ', '')}Regressor",
    task_types=[TaskType.REGRESSION],
    complexity=Complexity.MEDIUM,
    performance_metrics={
        'mse': results[best_model_name]['mse'],
        'mae': results[best_model_name]['mae'],
        'cv_mse': cv_mean,
        'cv_std': cv_std,
        'train_time': results[best_model_name]['train_time']
    },
    tags=['housing', 'california', 'regression', 'production-ready'],
    version='1.0.0'
)

# Registrar modelo
registry.register(metadata)
print(f"✅ Modelo registrado: {metadata.name}")
print(f"📋 Versión: {metadata.version}")
print(f"🏷️ Tags: {', '.join(metadata.tags)}")

# Buscar en el registro
print("\n🔍 Búsqueda en el registro...")
regression_models = registry.search(task_type=TaskType.REGRESSION)
print(f"📊 Modelos de regresión encontrados: {len(regression_models)}")

for model in regression_models:
    print(f"  • {model.display_name} (v{model.version})")

## 9️⃣ Comparación MLPY vs scikit-learn

In [ ]:
display(Markdown("### ⚔️ MLPY vs scikit-learn"))

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Preparar datos para sklearn
X_train = train_task.data.drop('MedHouseVal', axis=1)
y_train = train_task.data['MedHouseVal']
X_test = test_task.data.drop('MedHouseVal', axis=1)
y_test = test_task.data['MedHouseVal']

# Entrenar modelo sklearn
print("🔬 Entrenando Random Forest con scikit-learn...")
sklearn_rf = RandomForestRegressor(n_estimators=100, random_state=42)

start_time = time()
sklearn_rf.fit(X_train, y_train)
sklearn_train_time = time() - start_time

sklearn_pred = sklearn_rf.predict(X_test)
sklearn_mse = mean_squared_error(y_test, sklearn_pred)

# Crear tabla de comparación
comparison_df = pd.DataFrame({
    'Framework': ['MLPY', 'scikit-learn'],
    'MSE': [results['Random Forest']['mse'], sklearn_mse],
    'Training Time (s)': [results['Random Forest']['train_time'], sklearn_train_time]
})

display(Markdown("### 📊 Resultados de la Comparación"))
display(comparison_df)

# Ventajas de MLPY
display(HTML("""
<div style="background-color: #fff3e0; padding: 15px; border-radius: 10px; margin: 15px 0;">
    <h4>🌟 Ventajas de MLPY Demostradas</h4>
    <ul>
        <li>✅ <b>API Unificada:</b> Misma interfaz para todos los modelos</li>
        <li>✅ <b>Validación Integrada:</b> Detección automática de problemas en datos</li>
        <li>✅ <b>Model Registry:</b> Gestión y versionado de modelos</li>
        <li>✅ <b>Ensemble Fácil:</b> Creación trivial de métodos ensemble</li>
        <li>✅ <b>Resampling Nativo:</b> Cross-validation integrado</li>
        <li>✅ <b>Sistema de Medidas Rico:</b> Múltiples métricas con agregación</li>
        <li>✅ <b>Abstracción de Tareas:</b> Código más limpio y mantenible</li>
    </ul>
</div>
"""))

## 📊 Visualizaciones

In [ ]:
# Crear DataFrame con todos los resultados
results_df = pd.DataFrame({
    'Model': list(results.keys()) + ['Baseline'],
    'MSE': [r['mse'] for r in results.values()] + [baseline_mse],
    'MAE': [r['mae'] for r in results.values()] + [baseline_mae],
    'Training Time (s)': [r['train_time'] for r in results.values()] + [baseline_train_time]
})

results_df = results_df.sort_values('MSE')

# Crear visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Comparación de MSE
ax1 = axes[0, 0]
colors = ['#2ecc71' if m != 'Baseline' else '#e74c3c' for m in results_df['Model']]
bars = ax1.barh(results_df['Model'], results_df['MSE'], color=colors)
ax1.set_xlabel('Mean Squared Error', fontsize=12)
ax1.set_title('Comparación de Rendimiento (MSE)', fontsize=14, fontweight='bold')
ax1.axvline(x=baseline_mse, color='red', linestyle='--', alpha=0.5, label='Baseline')
ax1.legend()

# Añadir valores en las barras
for bar, val in zip(bars, results_df['MSE']):
    ax1.text(bar.get_width(), bar.get_y() + bar.get_height()/2, 
             f'{val:.3f}', ha='left', va='center', fontweight='bold')

# 2. Tiempo de entrenamiento
ax2 = axes[0, 1]
ax2.bar(range(len(results_df)), results_df['Training Time (s)'], 
        color=['#3498db' if m != 'Baseline' else '#95a5a6' for m in results_df['Model']])
ax2.set_xticks(range(len(results_df)))
ax2.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax2.set_ylabel('Tiempo (segundos)', fontsize=12)
ax2.set_title('Tiempo de Entrenamiento', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Actual vs Predicho (mejor modelo)
ax3 = axes[1, 0]
best_preds = results[results_df.iloc[0]['Model']]['predictions']
ax3.scatter(best_preds.truth, best_preds.response, alpha=0.5, s=10)
ax3.plot([0, 5], [0, 5], 'r--', lw=2, label='Predicción perfecta')
ax3.set_xlabel('Valores Reales', fontsize=12)
ax3.set_ylabel('Valores Predichos', fontsize=12)
ax3.set_title(f'Actual vs Predicho ({results_df.iloc[0]["Model"]})', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Resultados de Cross-Validation
ax4 = axes[1, 1]
bp = ax4.boxplot(cv_scores, patch_artist=True)
bp['boxes'][0].set_facecolor('#9b59b6')
ax4.set_ylabel('MSE', fontsize=12)
ax4.set_title('Distribución 5-Fold Cross-Validation', fontsize=14, fontweight='bold')
ax4.axhline(y=cv_mean, color='red', linestyle='--', 
            label=f'Media: {cv_mean:.4f}')
ax4.axhline(y=cv_mean + cv_std, color='orange', linestyle=':', alpha=0.5)
ax4.axhline(y=cv_mean - cv_std, color='orange', linestyle=':', alpha=0.5, 
            label=f'±1 std: {cv_std:.4f}')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle('MLPY Framework - Análisis de Resultados', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Tabla resumen de resultados
display(Markdown("### 📋 Ranking Final de Modelos"))

# Calcular mejoras
results_df['Improvement vs Baseline (%)'] = \
    ((baseline_mse - results_df['MSE']) / baseline_mse * 100).round(1)

# Estilizar DataFrame
styled_df = results_df.style.background_gradient(subset=['MSE'], cmap='RdYlGn_r') \
                           .background_gradient(subset=['MAE'], cmap='RdYlGn_r') \
                           .background_gradient(subset=['Improvement vs Baseline (%)'], cmap='RdYlGn') \
                           .format({'MSE': '{:.4f}', 
                                   'MAE': '{:.4f}', 
                                   'Training Time (s)': '{:.3f}',
                                   'Improvement vs Baseline (%)': '{:.1f}%'})

display(styled_df)

## 🎯 Conclusiones

### 📊 Resultados Clave

In [ ]:
# Calcular estadísticas finales
best_mse = results_df.iloc[0]['MSE']
best_model = results_df.iloc[0]['Model']
improvement = ((baseline_mse - best_mse) / baseline_mse) * 100

display(HTML(f"""
<div style="background-color: #d4edda; padding: 20px; border-radius: 10px; border-left: 5px solid #28a745;">
    <h3>🏆 Resumen de Resultados</h3>
    <table style="width: 100%; font-size: 14px;">
        <tr>
            <td><b>Mejor Modelo:</b></td>
            <td>{best_model}</td>
        </tr>
        <tr>
            <td><b>MSE Final:</b></td>
            <td>{best_mse:.4f}</td>
        </tr>
        <tr>
            <td><b>Mejora vs Baseline:</b></td>
            <td>{improvement:.1f}%</td>
        </tr>
        <tr>
            <td><b>CV MSE (±std):</b></td>
            <td>{cv_mean:.4f} ± {cv_std:.4f}</td>
        </tr>
        <tr>
            <td><b>Modelos Evaluados:</b></td>
            <td>{len(results)}</td>
        </tr>
    </table>
</div>
"""))

### 💡 Puntos Clave Demostrados

1. **🎯 API Unificada**: MLPY proporciona una interfaz consistente para todos los modelos
2. **🔍 Validación Automática**: Detección proactiva de problemas en los datos
3. **📚 Model Registry**: Gestión profesional de modelos con versionado
4. **🎭 Ensemble Methods**: Creación simple de modelos ensemble complejos
5. **📊 Evaluación Robusta**: Cross-validation y múltiples métricas integradas
6. **⚡ Rendimiento**: Comparable o superior a implementaciones directas
7. **🔧 Extensibilidad**: Fácil integración con el ecosistema Python ML

### 🚀 Próximos Pasos

- **AutoML**: Explorar capacidades de optimización automática
- **Deep Learning**: Integrar modelos de redes neuronales
- **Feature Engineering**: Pipeline automatizado de transformación
- **MLOps**: Deployment y monitoreo en producción
- **Distributed Training**: Escalar a datasets más grandes

---

### 📚 Recursos

- 📖 [Documentación Completa](../docs/DOCUMENTATION_SUMMARY.md)
- 🎓 [Tutoriales](../docs/tutoriales/00_INDICE_TUTORIALES.md)
- 💻 [GitHub Repository](https://github.com/your-org/mlpy)
- 📧 [Contacto](mailto:mlpy@example.com)

---

**¡Gracias por explorar MLPY Framework! 🚀**